# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [3]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [4]:
# TODO: Load environment variables
load_dotenv()

True

### VectorDB Instance

In [5]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="./chromadb")

### Collection

In [6]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base="https://openai.vocareum.com/v1"
)

In [7]:
# TODO: Create a collection
# Choose any name you want
collection = chroma_client.create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)

### Add documents

In [8]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [9]:
# Search Queries
questions = [
    "When was Grand Theft Auto released?",
    "When did Grand Theft Auto come out?",
    "Which platform was Grand Theft Auto released on?",
    "What games are available for Nintendo Switch?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?"
]

In [10]:
# Query DB for questions
for question in questions:
        print(f"Question: {question}")
        results = collection.query(
                query_texts=[question],
                n_results=3
        )
        for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                data = {
                        "Platform": meta.get("Platform", "Unknown"),
                        "Name": meta.get("Name", "Unknown"),
                        "YearOfRelease": meta.get("YearOfRelease", "Unknown"),
                        "Description": meta.get("Description", "N/A"),
                }

                print(json.dumps(data, indent=4))

Question: When was Grand Theft Auto released?
{
    "Platform": "PlayStation 2",
    "Name": "Grand Theft Auto: San Andreas",
    "YearOfRelease": 2004,
    "Description": "An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson."
}
{
    "Platform": "PlayStation 1",
    "Name": "Gran Turismo",
    "YearOfRelease": 1997,
    "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre."
}
{
    "Platform": "PlayStation 3",
    "Name": "Gran Turismo 5",
    "YearOfRelease": 2010,
    "Description": "A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics."
}
Question: When did Grand Theft Auto come out?
{
    "Platform": "PlayStation 2",
    "Name": "Grand Theft Auto: San Andreas",
    "YearOfRelease": 2004,
    "Description": "An expansive open-world game set in the fictional state of San Andreas, following t